### Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer as Stem
from nltk.stem import WordNetLemmatizer as Lem

### Importing DataSets

In [ ]:
fake = pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv')
true = pd.read_csv('../input/fake-and-real-news-dataset/True.csv')

In [ ]:
print(fake.shape, true.shape)

### Adding a label to classify the datasets

In [ ]:
zero = np.zeros(fake.shape[0])

In [ ]:
fake['label'] = zero

In [ ]:
fake.head()

In [ ]:
one = np.ones(true.shape[0])
true['label'] = one

In [ ]:
true.head()

### Merging the datasets and shuffling it

In [ ]:
dataset = pd.concat([fake,true])

In [ ]:
dataset = dataset.sample(frac=1)
dataset = dataset.reset_index()

In [ ]:
dataset.head()

In [ ]:
dataset.drop(['index'],axis=1,inplace=True)

### Checking if any nulls are present

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.head()

### Dividing the sataset into X and y(target)

In [ ]:
X = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

In [ ]:
X.head()

In [ ]:
y.head()

### Dropping the date as it is not necessary

In [ ]:
X.drop(['date'],axis=1,inplace=True)

### Splitting the dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
X_test.columns

In [ ]:
import re

### Applying porter stemmer and stopwords and removing all punctuations from the title in train

In [ ]:
ps = Stem()
train_corpus = []
for i in X_train.index:
    review = re.sub('[^a-zA-Z]',' ',X_train['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(i) for i in review if i not in stopwords.words('english')]
    review = ' '.join(review)
    train_corpus.append(review)

### Implementing Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB as MNB

### Applying Count Vectorizer(Bag of Words) in train

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))

In [ ]:
X_train = cv.fit_transform(train_corpus)

In [ ]:
model = MNB()
model.fit(X_train,y_train)

### Applying porter stemmer and stopwords and removing all punctuations from the title in test

In [ ]:
ps = Stem()
test_corpus = []
for i in X_test.index:
    review = re.sub('[^a-zA-Z]',' ',X_test['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(i) for i in review if i not in stopwords.words('english')]
    review = ' '.join(review)
    test_corpus.append(review)

### Applying Count Vectorizer(Bag of Words) in test

In [ ]:
X_test = cv.transform(test_corpus)

In [ ]:
pred = model.predict(X_test)

### Performance Metrics

In [ ]:
from sklearn.metrics import confusion_matrix as cm, classification_report as cr, accuracy_score as acc
print(cm(y_test,pred))
print(cr(y_test,pred))
print(acc(y_test,pred))

### Using Hyper parameter in Multinomial Naive Bayes

In [ ]:
accuracy = 0
for alpha in np.arange(start=0,stop=1,step=0.1):
    model = MNB(alpha=alpha)
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    temp_accuracy = round(acc(y_test,pred),3)
    if(accuracy>temp_accuracy):
        classifier = model
        print('accuracy',accuracy,'model with alpha value', alpha)
        print('\n')
    else:
        accuracy = temp_accuracy

### Applying Passive Aggressive Classifier with n-jobs=50

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
model = PassiveAggressiveClassifier(n_jobs=50)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix as cm, classification_report as cr, accuracy_score as acc
print(cm(y_test,pred))
print(cr(y_test,pred))
print(acc(y_test,pred))

### Checking for different values of n-jobs

In [ ]:
accuracy = 0
for jobs in np.arange(start=0,stop=1000,step=50):
    model = PassiveAggressiveClassifier(n_jobs=jobs)
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    temp_accuracy = round(acc(y_test,pred),3)
    if(accuracy>temp_accuracy):
        classifier = model
        print('accuracy',accuracy,'model with n_jobs value', jobs)
        print('\n')
    else:
        print('continue')
        accuracy = temp_accuracy